In [1]:
import gurobipy as gp

#Abrindo a intancia
def build_instance(file_name):
    print(f"BUILDING INSTANCE {file_name.split("/")[len(file_name.split("/"))-1]}")
    instance_file = open(file_name,"r")
    
    lines = [line.strip() for line in instance_file.readlines()]
    current_line = 0
    
    #Salvando quantidade de clientes e quantidade e lockers
    qty_customer = int(lines[current_line].split(" ")[0])
    qty_locker = int(lines[current_line].split(" ")[1])
    
    current_line += 1
    
    max_vehicle = int(lines[current_line].split(" ")[0])
    vehicle_capacity = float(lines[current_line].split(" ")[1])
    
    current_line += 1
    
    demands = list()
    ## Salvando demands de clients
    for demand in range(current_line,qty_customer+2):
        demands.append(float(lines[demand]))
    
    current_line += qty_customer
    
    #Preenchendo nos
    nodes = dict()
    
    #Deposito
    depot = {
        'label': "D",
        'x' : float(lines[current_line].split(" ")[0]),
        'y' : float(lines[current_line].split(" ")[1]),
        'earliest' : float(lines[current_line].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[current_line].split(" ")[4]),
        'demand': 0,
        'type' : int(lines[current_line].split(" ")[5])
    }
    nodes['D']=depot
    current_line += 1

    #Clientes
    customers = dict()
    customer_count = 1
    for i in range(current_line, current_line + qty_customer):
        customer = {
        'label' : f"C{customer_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': demands[customer_count-1],
        'type' : int(lines[i].split(" ")[5])
        }
        customer_count+=1
        customers[customer['label']]=customer
        nodes[customer['label']]=customer
    current_line += qty_customer

    #Lockers
    lockers = dict()
    locker_count = 1
    for i in range(current_line,current_line+qty_locker):
        locker = {
        'label' : f"P{locker_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': 0,
        'customers': list(),
        'type' : int(lines[i].split(" ")[5])
        }
        locker_count+=1
        lockers[locker['label']] = locker
        nodes[locker['label']]=locker
    current_line += qty_locker

    #Atribuição de lockers
    current_customer = 0
    for i in range(current_line, current_line+qty_customer):
        try:
            customer = customers[list(customers.keys())[current_customer]]
            
            locker_index = lines[i].split(" ").index("1")
            locker = lockers[list(lockers.keys())[locker_index]]
            customer['locker'] = locker['label']
            locker['customers'].append(customer['label'])
        except ValueError:    
            customer['locker'] = ""
        current_customer += 1
    current_line += qty_customer
    
    #Matriz de Distancia
    costs = dict()
    for i,node_i in enumerate(nodes):
         current_node_i = nodes[node_i]
         current_node_i['index'] = i
         for j,node_j in enumerate(nodes):
             current_node_j = nodes[node_j]
             distance = ((current_node_i['x']-current_node_j['x'])**2 + (current_node_i['y']-current_node_j['y'])**2)**(1/2)
             costs[current_node_i['label'],current_node_j['label']] = distance
    print(f"Qty customer:{qty_customer}")
    print(f"Qty locker:{qty_locker}")
    print(f"Max vehicles:{max_vehicle}")
    print(f"Vehicle capacity:{vehicle_capacity}")
    print(f"BUILD FINISHED")
    print(sum(demands))
    return {
        'qty_customer': qty_customer,
        'qty_locker': qty_locker,
        'qty_nodes': qty_locker,
        'total_demands': sum(demands),
        'min_vehicles': round(sum(demands)/vehicle_capacity),
        'max_vehicle': max_vehicle,
        'vehicle_capacity': vehicle_capacity,
        'nodes': nodes,
        'customers': customers,
        'lockers': lockers,
        'depot': depot,
        'costs': costs
    }

def print_labels(nodes, lockers, customers, customers_hc, customers_lc, customers_hlc, vehicles):
    print(f"Nodes: {nodes}")
    print(f"Depot: {depot}")
    print(f"Lockers: {lockers}")
    print(f"Customers: {customers}")
    print(f"Customers HC : {customers_hc}")
    print(f"Customers LC: {customers_lc}")
    print(f"Customers HLC: {customers_hlc}")
    print(f"Vehicles: {vehicles}")
    
instance = build_instance("../../instances/vrppl/inst_test")

BUILDING INSTANCE inst_test
Qty customer:10
Qty locker:2
Max vehicles:2
Vehicle capacity:100.0
BUILD FINISHED
150.0


In [2]:
qty_vehicles = 3
v_capacity = int(instance['vehicle_capacity'])
#Rótulos
nodes = list(instance['nodes'].keys())
lockers = [node for node in nodes if instance['nodes'][node]['type'] == 4]
customers = list(instance['customers'].keys())
customers_hc = [customer for customer in customers if instance['nodes'][customer]['type'] == 1]
customers_lc = [customer for customer in customers if instance['nodes'][customer]['type'] == 2]
customers_hlc = [customer for customer in customers if instance['nodes'][customer]['type'] == 3]
depot = [node for node in nodes if instance['nodes'][node]['type'] == 0][0]

edges = [(node_i,node_j) for node_i in nodes for node_j in nodes if node_i != node_j]
# print(edges)

#Veiculos
vehicles = list()
for i in range(qty_vehicles):
    vehicles.append(f"v_{i}")

print_labels(nodes, lockers, customers, customers_hc, customers_lc, customers_hlc, vehicles)


Nodes: ['D', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'P1', 'P2']
Depot: D
Lockers: ['P1', 'P2']
Customers: ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']
Customers HC : ['C7', 'C9', 'C10']
Customers LC: ['C1', 'C2', 'C3']
Customers HLC: ['C4', 'C5', 'C6', 'C8']
Vehicles: ['v_0', 'v_1', 'v_2']


In [3]:
#Criando dicts

#Nos
nodes_dict = instance['nodes']

#Custos
costs = instance['costs']

#Demandas
demands = dict()

for node in nodes_dict:
    demands[node] = nodes_dict[node]['demand']

#Atribuições de locker
locker_attrib = dict()

for locker in lockers:
    for customer in customers:
        if(customer in nodes_dict[locker]['customers']):
            locker_attrib[customer,locker] = 1
        else:
            locker_attrib[customer,locker] = 0


In [4]:
#Criando modelo
m = gp.Model()
m.setParam(gp.GRB.Param.OutputFlag,0)

#Criando Variaveis
x_ijk = m.addVars(edges, vehicles, vtype=gp.GRB.BINARY)
v_k = m.addVars(vehicles, vtype = gp.GRB.BINARY)
h_i = m.addVars(customers, vtype = gp.GRB.BINARY)
l_i = m.addVars(customers, vtype = gp.GRB.BINARY)
y_ij = m.addVars(customers,lockers, vtype = gp.GRB.BINARY)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2621941
Academic license 2621941 - for non-commercial use only - registered to ra___@gmail.com


In [5]:
#Função objetivo
m.setObjective(
    gp.quicksum(costs[node_i,node_j] * x_ijk[node_i,node_j,v] for node_i in nodes for node_j in nodes for v in vehicles if node_i != node_j)
)

#Restrições

# #Cliente hc apenas em casa
c2 = m.addConstrs(
    gp.quicksum(x_ijk[node_i,node_j,v] for node_j in nodes for v in vehicles if node_i != node_j) == 1 for node_i in customers_hc
)

#Não visitar cliente lc
c3 = m.addConstrs(
    gp.quicksum(x_ijk[node_i,node_j,v] for node_j in nodes for v in vehicles if node_i != node_j) <= h_i[node_i] for node_i in customers_hlc
)

#Não visitar cliente se for customer_lc
c4 = m.addConstrs(
    gp.quicksum(x_ijk[node_i,node_j,v] for node_j in nodes for v in vehicles if node_i != node_j) == 0 for node_i in customers_lc
)

#Sempre sair do deposito
c5 = m.addConstrs(
    gp.quicksum(x_ijk[depot,node_j,v] for node_j in customers if node_j != depot) == v_k[v] for v in vehicles
)

# #Dinâmica de fluxo
c6 = m.addConstrs(
    gp.quicksum(x_ijk[node_i,node_j,v] for node_i in nodes if node_i != node_j) ==
    gp.quicksum(x_ijk[node_j,node_i,v] for node_i in nodes if node_j != node_i) for node_j in nodes for v in vehicles  
    
)

#Sempre chiegar no deposito
c7 = m.addConstrs(
    gp.quicksum(x_ijk[node_i,depot,v] for node_i in customers if node_i != depot) == v_k[v] for v in vehicles
)


# #Limitação de capacidade do locker
c8 =  m.addConstrs(
    gp.quicksum(demands[node_j] * x_ijk[node_i,node_j,v] for node_i in nodes for node_j in customers if node_i != node_j) <= v_capacity * v_k[v]
    for v in vehicles
)

# --- #

#Ou casa ou locker vai atender
c13 =  m.addConstrs(
    h_i[node_i] + l_i[node_i] == 1 for node_i in customers
)

#h_i sempre 1 pra cliente hc
c14 =  m.addConstrs(
    h_i[node_i] == 1 for node_i in customers_hc
)

#l_i sempre 1 pra cliente lc
c15 =  m.addConstrs(
    l_i[node_i] == 1 for node_i in customers_lc
)

#
c16 =  m.addConstrs(
    gp.quicksum(y_ij[node_i,locker_j] for locker_j in lockers) == l_i[node_i] for node_i in customers
)

#Atribuição de lockers
c17 =  m.addConstrs(
    y_ij[node_i,locker_j] <= locker_attrib[node_i,locker_j] for node_i in customers for locker_j in lockers
)


c21 =  m.addConstrs(
    gp.quicksum(x_ijk[node_i,node_j,v] for node_i in nodes for v in vehicles if node_j != node_i) >= h_i[node_j] for node_j in customers
)


In [6]:
m.optimize()
print(f"Resultado final: {m.objVal}")

for vehicle in vehicles:
    print(vehicle,": ", end = " ")
    for node_i in nodes:
        for node_j in nodes:
            if(node_i != node_j):
                if round(x_ijk[node_i,node_j,vehicle].X) == 1:
                    print(f"{node_i},{node_j}", " ", end="")
                
    print('\n')

print("\nVariavel v_k")
for vehicle in vehicles:
    print(f"{vehicle}: {round(v_k[vehicle].X)}", end=" ")
print("\n\nVariavel h_i")
for i in customers:
    print(f"h_{i}= {round(h_i[i].X)}", end=" ")
print("\n\nVariavel l_i")
for i in customers:
    print(f"l_{i}= {round(l_i[i].X)}", end=" ")
print("\n\nVariavel y_ij")
for i in customers:
    for j in lockers:
        print(f"y_{i}{j}= {round(y_ij[i,j].X)}", end=" ")

Resultado final: 41.15298244508294
v_0 :  D,C7  C7,D  C8,C9  C9,C8  C10,P1  P1,C10  

v_1 :  

v_2 :  


Variavel v_k
v_0: 1 v_1: 0 v_2: 0 

Variavel h_i
h_C1= 0 h_C2= 0 h_C3= 0 h_C4= 0 h_C5= 0 h_C6= 0 h_C7= 1 h_C8= 1 h_C9= 1 h_C10= 1 

Variavel l_i
l_C1= 1 l_C2= 1 l_C3= 1 l_C4= 1 l_C5= 1 l_C6= 1 l_C7= 0 l_C8= 0 l_C9= 0 l_C10= 0 

Variavel y_ij
y_C1P1= 0 y_C1P2= 1 y_C2P1= 1 y_C2P2= 0 y_C3P1= 0 y_C3P2= 1 y_C4P1= 0 y_C4P2= 1 y_C5P1= 0 y_C5P2= 1 y_C6P1= 0 y_C6P2= 1 y_C7P1= 0 y_C7P2= 0 y_C8P1= 0 y_C8P2= 0 y_C9P1= 0 y_C9P2= 0 y_C10P1= 0 y_C10P2= 0 

In [7]:
for var in x_ijk:
    if round(x_ijk[var].X) == 1:
        print(var, round(x_ijk[var].X))

('D', 'C7', 'v_0') 1
('C7', 'D', 'v_0') 1
('C8', 'C9', 'v_0') 1
('C9', 'C8', 'v_0') 1
('C10', 'P1', 'v_0') 1
('P1', 'C10', 'v_0') 1
